Case scenario: Testing MCAT Questions

In [1]:
import sys
import os
path = os.path.abspath('..')
if path not in sys.path:
    sys.path.insert(0, path)

import collections
import concurrent.futures
import datetime

# from google.colab import widgets
from IPython import display

from concordia import components as generic_components
from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia.agents import basic_agent
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import measurements as measurements_lib
from concordia.utils import html as html_lib
from concordia.utils import plotting

import logging
logging.basicConfig(level=logging.ERROR, filename='components_testing.log')
logger = logging.getLogger('ollama')

In [2]:
# Setup sentence encoder
from sentence_transformers import SentenceTransformer
st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
embedder = st5_model.encode

In [3]:
from concordia.language_model import ollama_model
model = ollama_model.OllamaLanguageModel(
    # model_name='llama2:70b',
    model_name='llama2'
    # streaming=True
)

# import dotenv
# import os
# dotenv.load_dotenv()
# api_key = os.getenv('OPENAI_API_KEY')

# from concordia.language_model import gpt_model
# model = gpt_model.GptLanguageModel(
#     api_key=api_key,
#     model_name='gpt-4',
# )

In [4]:
#@title Make the clock
time_step = datetime.timedelta(minutes=20)
SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
clock = game_clock.MultiIntervalClock(
    start=SETUP_TIME,
    step_sizes=[time_step, datetime.timedelta(seconds=10)])

In [5]:
measurements = measurements_lib.Measurements()

In [8]:
import importlib
import custom_components.quiz_metric as qm
importlib.reload(qm)
from concordia.language_model import ollama_model
importlib.reload(ollama_model)
model = ollama_model.OllamaLanguageModel(
    # model_name='llama2:70b',
    model_name='llama2'
    # streaming=True
)

context = """John Doe, a distinguished lawyer, has pursued a career path singularly 
focused on law and history since his high school years. 
Demonstrating an early affinity for the intricacies of legal systems and the rich tapestry 
of human history, John excelled in these subjects, showcasing an exceptional ability to 
analyze complex legal texts and historical documents. This focus, however, came at the cost of 
his performance in other areas; notably, John struggled significantly with the sciences 
during his high school years. He found the logical and empirical frameworks of subjects 
like biology and mathematics challenging, ultimately failing to achieve passing grades 
in these classes. Despite these early academic setbacks, John's dedication to law and 
history never wavered. His deep interest in legal studies and historical analysis fueled 
his academic journey through college and law school, where he thrived in an environment that 
played to his strengths. John's unwavering focus on law and history has not only defined his 
educational path but also shaped his professional career, making him a lawyer with a unique 
perspective, deeply rooted in the understanding of historical contexts and legal precedents.

John Doe is taking the medical school entrance examination, just for fun, to test his knowledge. 
If John Doe doesn't know the answer, he will still guess and begin his answer with a single choice. 
How would John Doe answer the following question?
"""

quiz_metric = qm.QuizMetric(model, "John Doe", clock, "./custom_components/mcat_questions.json", measurements=measurements, verbose=True)
quiz_metric.observe(context)

Sample: (a) CAR T-cell therapy
Answer: a
Question: What recent development in cancer treatment involves modifying a patient's own T cells to attack cancer cells?
John Doe's answer: CAR T-cell therapy
Correct answer: CAR T-cell therapy

Sample: (b) Rivaroxaban
Answer: b
Question: Which of the following is a novel oral anticoagulant (NOAC) introduced in the last 20 years?
John Doe's answer: Rivaroxaban
Correct answer: Rivaroxaban

Sample: (c) Pre-exposure Prophylaxis
Answer: c
Question: In the context of HIV treatment, what does the term 'PrEP' stand for?
John Doe's answer: Pre-exposure Prophylaxis
Correct answer: Pre-exposure Prophylaxis

Sample: John Doe would answer (c) TNF-Alpha as a target of modern monoclonal antibody therapy in autoimmune diseases.
Answer: c
Question: Which of these is a target of modern monoclonal antibody therapy in autoimmune diseases?
John Doe's answer: TNF-Alpha
Correct answer: TNF-Alpha

Sample: John Doe would answer (c) Editing genes at specific locations.
